In [51]:
import psycopg2
import pandas as pd
import geopandas as gpd
import numpy as np
import json
from sqlalchemy import create_engine, text

In [4]:
# Define database connection parameters
database_connection = {
    'drivername': 'postgresql',
    'username': 'postgres',
    'password': 'postgres',
    'host': 'localhost',
    'port': '5432',
    'database': 'vic_db',
}


# A psycopg2 connection and cursor
conn = psycopg2.connect(user=database_connection['username'],
                        password=database_connection['password'],
                        host=database_connection['host'],
                        port=database_connection['port'],
                        database=database_connection['database'])
conn.autocommit = True
cursor = conn.cursor()

# Create a SQLAlchemy engine
engine = create_engine('postgresql://%(username)s:%(password)s@%(host)s/%(database)s' % database_connection, isolation_level="AUTOCOMMIT")
conn_alchemy = engine.connect()

In [41]:

# Define the SQL query to select data from the PostGIS table
sql = "SELECT * FROM vmtrans.tr_road_all LIMIT 10;"

# Read data from PostGIS into a GeoDataFrame
gdf = gpd.read_postgis(sql, con=engine)

# # Print the GeoDataFrame
# print(gdf)


In [9]:
# List all tables
cursor.execute("SELECT table_schema, table_name FROM information_schema.tables;")
result_df = pd.DataFrame(cursor.fetchall(), columns=[desc[0] for desc in cursor.description])

In [11]:
cursor.execute("SELECT * FROM vmtrans.tr_road_all LIMIT 10;")
result_df = pd.DataFrame(cursor.fetchall(), columns=[desc[0] for desc in cursor.description])

In [37]:
# In table tr_road_all, create a column called "road_length" which is the length of the road segment
# But not really create it, just create a new column from SELECT statement. Use meters as unit, so we have to convert the length from degree to meters
# cursor.execute("SELECT SUM(ST_Length(geom::geography)) as road_length FROM vmtrans.tr_road_all WHERE ezi_road_name LIKE '%GLENBROOK AVENUE%';")
cursor.execute(
"""
SELECT 
    ST_Length(geom::geography) as road_length,
    ST_AsText(ST_StartPoint(ST_GeometryN(geom, 1))) AS first_point,
    ST_AsText(ST_EndPoint(ST_GeometryN(geom, 1))) AS last_point
FROM vmtrans.tr_road_all 
WHERE ezi_road_name LIKE '%GLENBROOK AVENUE%';
""")
result_df = pd.DataFrame(cursor.fetchall(), columns=[desc[0] for desc in cursor.description])
result_df

,road_length,first_point,last_point
0,74.321053,POINT(144.7845466330001 -37.765636496999946),POINT(144.78466143100002 -37.76497362999993)
1,114.023511,POINT(145.0549611360001 -37.86946304899993),POINT(145.05515306400002 -37.86844708399997)
2,271.300323,POINT(145.12387210600002 -37.90911573999995),POINT(145.12431451100008 -37.90669674099996)
3,52.212317,POINT(145.0548732430001 -37.86992826699998),POINT(145.0549611360001 -37.86946304899993)
4,247.812288,POINT(144.78573093800003 -37.76742263799997),POINT(144.7845466330001 -37.765636496999946)
5,13.516021,POINT(145.12301246200002 -37.91167085699993),POINT(145.12310854800012 -37.911575813999946)
6,287.039748,POINT(145.12310854800012 -37.911575813999946),POINT(145.12387210600002 -37.90911573999995)
7,17.342869,POINT(145.1290169560001 -38.05468524999998),POINT(145.12883521900005 -38.05474658899993)
8,301.855147,POINT(145.05515306400002 -37.86844708399997),POINT(145.0556611290001 -37.86575751099997)
9,164.729226,POINT(144.7879104740001 -37.76769546799994),POINT(144.7860614210001 -37.76747599199996)


In [59]:
# sql = """
# SELECT ezi_road_name, ST_Length(geom::geography) as road_length, ST_AsText(ST_StartPoint(ST_GeometryN(geom, 1))) AS first_point, ST_AsText(ST_EndPoint(ST_GeometryN(geom, 1))) AS last_point FROM vmtrans.tr_road_all WHERE ezi_road_name LIKE '%GLENBROOK AVENUE%';
# """
sql = """
SELECT
    GeometryType(geom) AS geometry_type,
    COUNT(*) AS count
FROM vmtrans.tr_road_all
GROUP BY GeometryType(geom);
"""

pd.read_sql_query(text(sql), conn_alchemy)

,geometry_type,count
0,MULTILINESTRING,1222415


In [63]:
pd.read_sql_query(text("SELECT * FROM vmtrans.tr_road_all LIMIT 10;"), conn_alchemy)

,gid,ufi,pfi,feature_type_code,named_feature_id,ezi_road_name,ezi_road_name_label,road_name,road_type,road_suffix,...,to_ufi,feature_quality_id,task_id,create_date_pfi,superceded_pfi,create_date_ufi,geom,road_length_meters,first_point,last_point
0,115,1383777.0,5672732.0,road,None,WINMALEE ROAD,Winmalee Road,WINMALEE,ROAD,None,...,2309464.0,1328.0,NaN,2000-12-17,NaN,2000-12-17,0105000020A41E00000100000001020000000200000008...,149.992377,0101000020A41E000008EC03C613236240804D8D6574E7...,0101000020A41E00002CE4109221236240407A4A017BE7...
1,20,16432344.0,6315852.0,road,None,CREEK STREET N,Creek Street N,CREEK,STREET,N,...,16432884.0,4001.0,1960.0,2001-12-21,0.0,2006-11-03,0105000020A41E00000100000001020000000200000024...,86.459490,0101000020A41E000024E3B90D96086240705DC3CF2962...,0101000020A41E00002C5444419108624060380E243E62...
2,50,19190562.0,5793379.0,road,None,HOTHAM ROAD,Hotham Road,HOTHAM,ROAD,None,...,15514697.0,1347.0,4.0,2000-12-17,0.0,2008-05-26,0105000020A41E0000010000000102000000020000004C...,25.851123,0101000020A41E00004C063E0EAC16624008873AE65B2A...,0101000020A41E0000D4DAABC4A9166240E896F663592A...
3,91,1348045.0,5637040.0,road,None,BRUCE STREET,Bruce Street,BRUCE,STREET,None,...,2277143.0,257.0,NaN,2000-12-17,NaN,2000-12-17,0105000020A41E00000100000001020000000200000010...,105.764756,0101000020A41E0000105D95B3831F624060C55420D1DE...,0101000020A41E000078D797748D1F624010C6E900D5DE...
4,1,15947521.0,5788298.0,road,None,TUCKFIELD STREET,Tuckfield Street,TUCKFIELD,STREET,None,...,2439077.0,1480.0,1826.0,2000-12-17,0.0,2005-10-21,0105000020A41E00000100000001020000000300000098...,96.382543,0101000020A41E000098883B0714116240F8875F771B22...,0101000020A41E000078451CF7151162407025591F0022...
5,92,1347232.0,5636228.0,road,None,MURRAY STREET,Murray Street,MURRAY,STREET,None,...,2276339.0,257.0,NaN,2000-12-17,NaN,2000-12-17,0105000020A41E0000010000000102000000020000003C...,87.245882,0101000020A41E00003CB1FC03B91E62402085D04690DE...,0101000020A41E00008062B30FC11E6240D00C4A7A93DE...
6,5,16325109.0,5899747.0,road,None,ECHUCA-MITIAMO ROAD,Echuca - Mitiamo Road,ECHUCA-MITIAMO,ROAD,None,...,2036057.0,4010.0,1904.0,2000-12-18,0.0,2006-07-25,0105000020A41E000001000000010200000002000000C8...,291.166852,0101000020A41E0000C84A9F323B146240C89564C45517...,0101000020A41E00009852A4B655146240682409C25517...
7,6,16287587.0,11789109.0,trail,None,UNNAMED,Unnamed,UNNAMED,None,None,...,16288188.0,4991.0,1879.0,2006-05-09,10087480.0,2006-05-09,0105000020A41E000001000000010200000004000000F4...,51.779561,0101000020A41E0000F4D2C728711F6240B04649B7C1EA...,0101000020A41E0000E4F47A6C751F6240604E31E3BAEA...
8,49,18459275.0,13174819.0,road,None,UNNAMED,Unnamed,UNNAMED,None,None,...,18459580.0,5627.0,2158.0,2007-08-07,NaN,2007-08-07,0105000020A41E0000010000000102000000020000001C...,11.162881,0101000020A41E00001C41B926AA276240F02743B87639...,0101000020A41E00004867AF2AAA2762401887C76C7339...
9,21,16725364.0,11810141.0,road,None,RACECOURSE TRACK,Racecourse Track,RACECOURSE,TRACK,None,...,16384630.0,5551.0,1648.0,2006-09-08,0.0,2007-04-17,0105000020A41E00000100000001020000000300000074...,118.994863,0101000020A41E000074C06751A30A6240B89BC1210CB3...,0101000020A41E0000C0B06A4DA90A6240505B462429B3...


In [62]:
# Add new columns to the table
sql = """
ALTER TABLE vmtrans.tr_road_all
ADD COLUMN road_length_meters double precision,
ADD COLUMN first_point geometry(Point, 7844),
ADD COLUMN last_point geometry(Point, 7844);

UPDATE vmtrans.tr_road_all
SET road_length_meters = ST_Length(geom::geography),
    first_point = ST_StartPoint(ST_GeometryN(geom, 1)),
    last_point = ST_EndPoint(ST_GeometryN(geom, 1));
"""
cursor.execute(sql)
# 1m 30s

DuplicateColumn: column "road_length_meters" of relation "tr_road_all" already exists
